In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
data = pd.read_csv('../ParticipantData_BTPC/Train.csv')

In [ ]:
data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
data.client.value_counts()

In [8]:
# Create a label encoder object 
def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        labelencoder_X=preprocessing.LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])
    return dataframe


def preprocess(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    cols = ['session_number','client','purchased','added_in_cart','checked_out','device','browser','time_spent'] # ,'year','month','day', time_spent
    inn = data[cols]
    columnlist = ['device','browser','client']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

pre_data = preprocess(data)


C:\Users\52052723\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
scaler = MinMaxScaler(feature_range=(0.01, 5))
scaled = scaler.fit_transform(pre_data)
scal = pd.DataFrame(data = scaled, columns = pre_data.columns)

Index(['session_number', 'client', 'purchased', 'added_in_cart', 'checked_out',
       'device', 'browser', 'time_spent'],
      dtype='object')
(5429, 8)


In [25]:
def model():
    y = scal[['time_spent']]
    x = scal.drop(['time_spent'], axis = 1)
    x_value = np.array(x)
    y_value = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(x_value, y_value,test_size=0.2,random_state=42)
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    predd = lin_reg.predict(X_test)
    print(mean_squared_log_error(predd, y_test)) 
    return lin_reg
lin_reg = model()

0.017876784383092936


In [14]:
def preprocess_test(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    cols = ['session_number','client','purchased','added_in_cart','checked_out','device','browser'] # ,'year','month','day', time_spent
    inn = data[cols]
    columnlist = ['device','browser','client']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

In [26]:
test = pd.read_csv('../ParticipantData_BTPC/Test.csv')
sub = pd.read_csv("../ParticipantData_BTPC/Sample Submission.csv")
pre_test = preprocess_test(test)
scaler = MinMaxScaler(feature_range=(0.01, 5))
scaled_test = scaler.fit_transform(pre_test)

(2327, 8)
(2327, 1)
7


C:\Users\52052723\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [27]:
pred = lin_reg.predict(scaled_test)


array([[0.15419901],
       [0.05316298],
       [0.18202075],
       ...,
       [0.12036657],
       [0.17020481],
       [0.15045105]])

In [28]:
sub['time_spent'] = pred
sub.to_csv("../ParticipantData_BTPC/Submission.csv", index = False)